# Bitcoin Investment Advisor Dataset

This notebook creates a comprehensive investment advisory dataset that:
1. Takes all daily news analysis data
2. Generates full market summaries using your OpenAI-compatible gateway
3. Provides detailed buy/sell/hold recommendations
4. Creates training data for investment advisory models

Models available via your gateway (tool-calling disabled):
- DeepSeek-V3.1
- Qwen3-32B
- Qwen2.5-72B (default)
- gemma-3-27b-it
- Llama4-Scout-17B-16E

Connection:
- base_url: https://gw.ai-platform.ir/v1
- api_key: set in env as AI_PLATFORM_API_KEY

In [1]:
# Install required packages
# If these are already installed in your environment, you can skip this cell.
%pip -q install --upgrade openai pandas numpy datasets huggingface_hub tqdm python-dateutil yfinance

import os
import pandas as pd
import numpy as np
import json
import glob
from datetime import datetime, timedelta
import time
import random
from typing import List, Dict, Any, Optional
from tqdm.notebook import tqdm
from openai import OpenAI
from datasets import Dataset
from huggingface_hub import login
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import yfinance as yf

print("✅ All required libraries imported successfully!")


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ All required libraries imported successfully!


In [2]:
# ===== API CONNECTION CONFIGURATION =====
# Connection to your OpenAI-compatible gateway
OPENAI_API_KEY = os.environ.get("AI_PLATFORM_API_KEY", "sk-SVSNSJKVosankQ4kFjl1Qg")
OPENAI_BASE_URL = os.environ.get("AI_PLATFORM_BASE_URL", "https://gw.ai-platform.ir/v1")

# Choose a model available in your gateway
# Options: "DeepSeek-V3.1", "Qwen3-32B", "Qwen2.5-72B", "gemma-3-27b-it", "Llama4-Scout-17B-16E"
DEFAULT_MODEL = os.environ.get("AI_PLATFORM_MODEL", "DeepSeek-V3.1")

# Optional: login to HuggingFace if you plan to push datasets (leave blank to skip)
HF_TOKEN = os.environ.get("HF_TOKEN", "")

# Initialize client
client = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)

# Test connection by listing models (safe, non-sensitive)
try:
    models = client.models.list()
    names = [m.id for m in models.data]
    print(f"✅ Connected to gateway. {len(names)} models visible. Example: {names[:5]}")
except Exception as e:
    print(f"⚠️ Could not list models: {e}")

# Login to HuggingFace if token provided
if HF_TOKEN:
    try:
        login(token=HF_TOKEN, add_to_git_credential=False)
        print("✅ HuggingFace login successful!")
    except Exception as e:
        print(f"⚠️ HuggingFace login failed: {e}")

✅ Connected to gateway. 5 models visible. Example: ['DeepSeek-V3.1', 'Qwen3-32B', 'Qwen2.5-72B', 'gemma-3-27b-it', 'Llama4-Scout-17B-16E']


# 2. Load and Aggregate All News Data

Load all daily news analysis and aggregate them for comprehensive investment advisory generation.

In [3]:
# Load all daily news analysis data with synthetic next 10-day and next 60-day prices
print("📰 Loading comprehensive daily news analysis with synthetic price predictions...")

def generate_realistic_next_10_day_prices(base_price: float = None, market_sentiment: str = "neutral") -> List[float]:
    """Generate realistic next 10-day Bitcoin prices based on market sentiment"""
    if base_price is None:
        # Use a realistic Bitcoin price range for the dataset period (2018-2024)
        base_price = np.random.uniform(8000, 70000)
    
    prices = [base_price]
    
    # Adjust volatility and trend based on sentiment
    if market_sentiment == "bullish":
        daily_trend = np.random.uniform(0.005, 0.03)  # 0.5% to 3% daily growth
        volatility = 0.04  # 4% daily volatility
    elif market_sentiment == "bearish":
        daily_trend = np.random.uniform(-0.03, -0.005)  # -3% to -0.5% daily decline
        volatility = 0.05  # 5% daily volatility (higher in bear markets)
    else:  # neutral
        daily_trend = np.random.uniform(-0.01, 0.01)  # -1% to +1% daily change
        volatility = 0.035  # 3.5% daily volatility
    
    for i in range(1, 10):
        # Add trend + random walk
        random_change = np.random.normal(daily_trend, volatility)
        new_price = prices[-1] * (1 + random_change)
        
        # Ensure price doesn't go below $1000 or above $150,000
        new_price = max(1000, min(150000, new_price))
        prices.append(new_price)
    
    return prices[1:]  # Return only the next 10 days, not the base price

def generate_realistic_next_60_day_prices(base_price: float = None, market_sentiment: str = "neutral") -> List[float]:
    """Generate realistic next 60-day Bitcoin prices for better analysis (hidden from training prompts)"""
    if base_price is None:
        base_price = np.random.uniform(8000, 70000)
    
    prices = [base_price]
    
    # Adjust volatility and trend based on sentiment for longer term
    if market_sentiment == "bullish":
        daily_trend = np.random.uniform(0.003, 0.02)  # 0.3% to 2% daily growth (slightly lower for 60 days)
        volatility = 0.045  # Slightly higher volatility over longer period
    elif market_sentiment == "bearish":
        daily_trend = np.random.uniform(-0.025, -0.003)  # -2.5% to -0.3% daily decline
        volatility = 0.055  # Higher volatility in bear markets over time
    else:  # neutral
        daily_trend = np.random.uniform(-0.008, 0.008)  # -0.8% to +0.8% daily change
        volatility = 0.04  # 4% daily volatility
    
    # Add some longer-term cycles and trend changes
    for i in range(1, 61):
        # Add cyclical behavior every 15-20 days
        cycle_factor = 1 + 0.1 * np.sin(i * np.pi / 15)
        
        # Gradually reduce trend strength over time (mean reversion)
        trend_decay = max(0.5, 1 - (i / 120))  # Trend decays over 60 days
        adjusted_trend = daily_trend * trend_decay * cycle_factor
        
        # Add random walk with trend
        random_change = np.random.normal(adjusted_trend, volatility)
        new_price = prices[-1] * (1 + random_change)
        
        # Ensure price doesn't go below $1000 or above $200,000
        new_price = max(1000, min(200000, new_price))
        prices.append(new_price)
    
    return prices[1:]  # Return only the next 60 days, not the base price

news_data_path = "/Users/tahamajs/Documents/uni/LLM/Files/Final Project/outputs_btc_effects/per_date/*.json"
all_news_files = sorted(glob.glob(news_data_path))

if not all_news_files:
    print("❌ No news files found. Please check the path.")
    print("Expected path:", news_data_path)
else:
    print(f"📁 Found {len(all_news_files)} news files")

# Load all comprehensive daily data for investment advisory
daily_investment_data = []

for file_path in tqdm(all_news_files[:], desc="Loading daily investment data"):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        date_str = os.path.basename(file_path).split('.')[0]

        # Extract existing data structure
        long_term_items = data.get('long_term', [])
        short_term_items = data.get('short_term', [])
        all_items = long_term_items + short_term_items
        daily_view = data.get('daily_view', {})

        # Determine market sentiment for price generation
        bull_prob = daily_view.get('scenario_probs', {}).get('bull', 0.33)
        bear_prob = daily_view.get('scenario_probs', {}).get('bear', 0.33)
        
        if bull_prob > 0.5:
            market_sentiment = "bullish"
        elif bear_prob > 0.5:
            market_sentiment = "bearish"
        else:
            market_sentiment = "neutral"

        # Generate realistic prices based on market sentiment
        # Use different base prices depending on the date to be more realistic
        try:
            year = int(date_str.split('-')[0])
            if year <= 2018:
                base_price_range = (3000, 20000)
            elif year <= 2020:
                base_price_range = (4000, 30000)
            elif year <= 2021:
                base_price_range = (10000, 70000)
            else:
                base_price_range = (15000, 50000)
            
            base_price = np.random.uniform(*base_price_range)
        except:
            base_price = 25000  # Default fallback
        
        # Generate both 10-day and 60-day price predictions
        next_10_day_prices = generate_realistic_next_10_day_prices(base_price, market_sentiment)
        next_60_day_prices = generate_realistic_next_60_day_prices(base_price, market_sentiment)

        daily_data = {
            'date': date_str,
            'daily_view': daily_view,
            'long_term_news': long_term_items,
            'short_term_news': short_term_items,
            'next_10_day_prices': next_10_day_prices,  # For training prompts
            'next_60_day_prices': next_60_day_prices,  # Hidden from training, used for better analysis
        }

        # Stats (keeping existing logic)
        total_items = len(all_items)
        bullish_count = sum(1 for item in all_items if item.get('direction', '').lower() == 'bullish')
        bearish_count = sum(1 for item in all_items if item.get('direction', '').lower() == 'bearish')
        confidences = [item.get('confidence', 0) for item in all_items if isinstance(item.get('confidence', 0), (int, float))]

        daily_data['total_news_items'] = total_items
        daily_data['long_term_count'] = len(long_term_items)
        daily_data['short_term_count'] = len(short_term_items)
        daily_data['bullish_ratio'] = bullish_count / total_items if total_items else 0
        daily_data['bearish_ratio'] = bearish_count / total_items if total_items else 0
        daily_data['neutral_ratio'] = (total_items - bullish_count - bearish_count) / total_items if total_items else 0
        daily_data['avg_confidence'] = float(np.mean(confidences)) if confidences else 0

        high_impact_news = [item for item in all_items if str(item.get('magnitude', '')).lower() == 'high']
        daily_data['high_impact_count'] = len(high_impact_news)
        daily_data['high_impact_news'] = high_impact_news[:5]

        daily_investment_data.append(daily_data)

    except Exception as e:
        print(f"⚠️ Error loading {file_path}: {e}")
        continue

# To DataFrame
df_daily_investment = pd.DataFrame(daily_investment_data)

if not df_daily_investment.empty:
    # keep date as datetime for sorting and readable printing
    df_daily_investment['date'] = pd.to_datetime(df_daily_investment['date'])
    df_daily_investment = df_daily_investment.sort_values('date').reset_index(drop=True)

    print(f"✅ Loaded {len(df_daily_investment)} daily investment datasets with synthetic next 10-day and 60-day price predictions")
    print(f"📅 Date range: {df_daily_investment['date'].min().date()} to {df_daily_investment['date'].max().date()}")

    print(f"\n📊 Daily Investment Data Statistics:")
    print(f"   Average news items per day: {df_daily_investment['total_news_items'].mean():.1f}")
    print(f"   Average high impact news per day: {df_daily_investment['high_impact_count'].mean():.1f}")
    print(f"   Average confidence: {df_daily_investment['avg_confidence'].mean():.2f}")

    # Display sample with both 10-day and 60-day prices
    print(f"\n📄 Sample daily data with synthetic price predictions:")
    sample_idx = len(df_daily_investment) // 2
    sample = df_daily_investment.iloc[sample_idx]
    print(f"Date: {sample['date'].date()}")
    print(f"Total news: {sample['total_news_items']}")
    print(f"High impact: {sample['high_impact_count']}")
    print(f"Bullish ratio: {sample['bullish_ratio']:.2f}")
    print(f"Avg confidence: {sample['avg_confidence']:.2f}")
    
    # Show market sentiment and prices
    daily_view = sample['daily_view']
    bull_prob = daily_view.get('scenario_probs', {}).get('bull', 0)
    bear_prob = daily_view.get('scenario_probs', {}).get('bear', 0)
    print(f"Market scenarios - Bull: {bull_prob:.1%}, Bear: {bear_prob:.1%}")
    
    # Show 10-day prices (for training prompts)
    print(f"Next 10-day prices (training): ${sample['next_10_day_prices'][0]:.2f}, ${sample['next_10_day_prices'][1]:.2f}, ${sample['next_10_day_prices'][2]:.2f}...")
    print(f"10-day price change: {((sample['next_10_day_prices'][-1] / sample['next_10_day_prices'][0]) - 1) * 100:.2f}%")
    
    # Show 60-day prices (hidden from training, for better analysis)
    print(f"Next 60-day prices (analysis): ${sample['next_60_day_prices'][0]:.2f}, ${sample['next_60_day_prices'][9]:.2f}, ${sample['next_60_day_prices'][29]:.2f}, ${sample['next_60_day_prices'][59]:.2f}")
    print(f"60-day price change: {((sample['next_60_day_prices'][-1] / sample['next_60_day_prices'][0]) - 1) * 100:.2f}%")
else:
    print("❌ No daily investment data loaded")

📰 Loading comprehensive daily news analysis with synthetic price predictions...
📁 Found 2437 news files


Loading daily investment data:   0%|          | 0/2437 [00:00<?, ?it/s]

✅ Loaded 2437 daily investment datasets with synthetic next 10-day and 60-day price predictions
📅 Date range: 2018-01-01 to 2024-12-31

📊 Daily Investment Data Statistics:
   Average news items per day: 20.2
   Average high impact news per day: 5.9
   Average confidence: 0.69

📄 Sample daily data with synthetic price predictions:
Date: 2021-07-16
Total news: 21
High impact: 5
Bullish ratio: 0.57
Avg confidence: 0.68
Market scenarios - Bull: 45.0%, Bear: 20.0%
Next 10-day prices (training): $55461.08, $57026.97, $50880.24...
10-day price change: -11.41%
Next 60-day prices (analysis): $56724.07, $51778.89, $46967.93, $59887.78
60-day price change: 5.58%


# 3. Create Investment Advisory Generator

Build a system using Gemini to generate comprehensive investment advice based on all daily news analysis.

In [4]:
class BitcoinInvestmentAdvisor:
    def __init__(self, api_key, base_url="https://gw.ai-platform.ir/v1", model="deepseek-chat"):
        # Store for diagnostics
        self.api_key = api_key
        self.base_url = base_url
        
        # OpenAI-compatible client
        self.client = OpenAI(api_key=api_key, base_url=base_url)
        self.model = model
        
    def analyze_price_movements_for_better_advisory(self, next_10_day_prices, next_60_day_prices):
        """Analyze price movements to create better advisory responses (hidden from training prompts)"""
        
        # 10-day analysis
        price_10d_change = ((next_10_day_prices[-1] / next_10_day_prices[0]) - 1) * 100
        price_10d_volatility = np.std([((next_10_day_prices[i] / next_10_day_prices[i-1]) - 1) * 100 
                                      for i in range(1, len(next_10_day_prices))])
        
        # 60-day analysis  
        price_60d_change = ((next_60_day_prices[-1] / next_60_day_prices[0]) - 1) * 100
        price_60d_volatility = np.std([((next_60_day_prices[i] / next_60_day_prices[i-1]) - 1) * 100 
                                      for i in range(1, len(next_60_day_prices))])
        
        # Weekly analysis (7, 14, 21, 28 days from 60-day data)
        weekly_changes = []
        for week in [6, 13, 20, 27]:  # 7, 14, 21, 28 days (0-indexed)
            if week < len(next_60_day_prices):
                weekly_change = ((next_60_day_prices[week] / next_60_day_prices[0]) - 1) * 100
                weekly_changes.append(weekly_change)
        
        # Monthly analysis (30 and 60 days)
        monthly_changes = []
        for month in [29, 59]:  # 30 and 60 days (0-indexed)
            if month < len(next_60_day_prices):
                monthly_change = ((next_60_day_prices[month] / next_60_day_prices[0]) - 1) * 100
                monthly_changes.append(monthly_change)
        
        # Determine market phases and trends based on actual future performance
        trend_strength = "weak"
        if abs(price_60d_change) > 30:
            trend_strength = "very strong"
        elif abs(price_60d_change) > 15:
            trend_strength = "strong" 
        elif abs(price_60d_change) > 5:
            trend_strength = "moderate"
        
        volatility_assessment = "low"
        if price_60d_volatility > 8:
            volatility_assessment = "very high"
        elif price_60d_volatility > 5:
            volatility_assessment = "high"
        elif price_60d_volatility > 3:
            volatility_assessment = "moderate"
        
        # Determine optimal strategy based on actual future performance
        if price_60d_change > 20:
            optimal_strategy = "aggressive buy and hold"
            risk_level = "high reward potential"
        elif price_60d_change > 10:
            optimal_strategy = "strategic accumulation"
            risk_level = "moderate-high"
        elif price_60d_change > -10:
            optimal_strategy = "selective buying on dips"
            risk_level = "moderate"
        elif price_60d_change > -25:
            optimal_strategy = "defensive positioning"
            risk_level = "moderate-high"
        else:
            optimal_strategy = "risk-off, minimal exposure"
            risk_level = "very high"
        
        return {
            'trend_strength': trend_strength,
            'volatility_assessment': volatility_assessment,
            'optimal_strategy': optimal_strategy,
            'risk_level': risk_level,
            'weekly_performance_pattern': weekly_changes,
            'monthly_performance_pattern': monthly_changes,
            '10d_expected_return': price_10d_change,
            '60d_expected_return': price_60d_change,
        }
        
    def create_investment_advisory_prompt(self, daily_data):
        """Create comprehensive investment advisory prompt using existing data structure with next 10-day prices ONLY"""
        newline = chr(10)
        
        date_str = daily_data['date'].strftime('%Y-%m-%d') if hasattr(daily_data['date'], 'strftime') else str(daily_data['date'])
        daily_view = daily_data['daily_view']
        long_term_news = daily_data['long_term_news']
        short_term_news = daily_data['short_term_news']
        next_10_day_prices = daily_data['next_10_day_prices']
        # NOTE: next_60_day_prices are NOT included in training prompts - they're used for better analysis
        
        # Format news summaries
        long_term_summary = ""
        for i, news in enumerate(long_term_news[:8], 1):  # Top 8 long-term news
            long_term_summary += f"{i}. {news.get('title', 'N/A')}{newline}"
            long_term_summary += f"   Impact: {news.get('direction', 'N/A')} ({news.get('magnitude', 'N/A')} magnitude, {news.get('confidence', 'N/A')} confidence){newline}"
            long_term_summary += f"   Horizon: {news.get('impact_horizon_months', 'N/A')} months{newline}"
            long_term_summary += f"   Summary: {news.get('summary', 'N/A')[:200]}...{newline}"
            long_term_summary += f"   Rationale: {news.get('rationale', 'N/A')}{newline}{newline}"
        
        short_term_summary = ""
        for i, news in enumerate(short_term_news[:8], 1):  # Top 8 short-term news
            short_term_summary += f"{i}. {news.get('title', 'N/A')}{newline}"
            short_term_summary += f"   Impact: {news.get('direction', 'N/A')} ({news.get('magnitude', 'N/A')} magnitude, {news.get('confidence', 'N/A')} confidence){newline}"
            short_term_summary += f"   Horizon: {news.get('impact_horizon_days', 'N/A')} days{newline}"
            short_term_summary += f"   Summary: {news.get('summary', 'N/A')[:200]}...{newline}"
            short_term_summary += f"   Rationale: {news.get('rationale', 'N/A')}{newline}{newline}"
        
        # Format market scenarios
        scenarios = daily_view.get('scenario_probs', {})
        bull_prob = scenarios.get('bull', 0)
        bear_prob = scenarios.get('bear', 0)
        base_prob = scenarios.get('base', 0)
        
        # Format recommendations
        short_rec = daily_view.get('recommendation_short_term', {})
        long_rec = daily_view.get('recommendation_long_term', {})
        
        # Format next 10-day price predictions (ONLY 10-day prices in training prompts)
        price_predictions = ""
        for i, price in enumerate(next_10_day_prices, 1):
            price_predictions += f"Day {i}: ${price:.2f}{newline}"
        
        price_change_10d = ((next_10_day_prices[-1] / next_10_day_prices[0]) - 1) * 100
        
        prompt = f"""You are an elite Bitcoin investment advisor with deep expertise in cryptocurrency markets, institutional trading strategies, and comprehensive financial analysis. Provide an extensive, institutional-grade investment advisory for Bitcoin based on the comprehensive market intelligence below.

DATE: {date_str}

MARKET INTELLIGENCE SUMMARY:
• Total News Items Analyzed: {daily_data['total_news_items']}
• Long-term Impact News: {daily_data['long_term_count']} items
• Short-term Impact News: {daily_data['short_term_count']} items
• High Impact News: {daily_data['high_impact_count']} items
• Market Sentiment Distribution: {daily_data['bullish_ratio']:.1%} Bullish, {daily_data['bearish_ratio']:.1%} Bearish, {daily_data['neutral_ratio']:.1%} Neutral
• Average Analyst Confidence: {daily_data['avg_confidence']:.2%}

NEXT 10-DAY PRICE PREDICTIONS:
{price_predictions}
Total 10-Day Price Change: {price_change_10d:+.2f}%

MARKET SCENARIO PROBABILITIES:
• Bullish Scenario: {bull_prob:.1%}
• Base Case Scenario: {base_prob:.1%}  
• Bearish Scenario: {bear_prob:.1%}

CURRENT MARKET RECOMMENDATIONS:
• Short-term Action: {short_rec.get('action', 'N/A')} (Probability: {short_rec.get('probability', 0):.1%})
• Long-term Action: {long_rec.get('action', 'N/A')} (Probability: {long_rec.get('probability', 0):.1%})

LONG-TERM IMPACT NEWS ANALYSIS:
{long_term_summary}

SHORT-TERM IMPACT NEWS ANALYSIS:
{short_term_summary}

KEY MARKET RISKS:
{newline.join(f"• {risk}" for risk in daily_view.get('key_risks', []))}

CRITICAL WATCH ITEMS:
{newline.join(f"• {item}" for item in daily_view.get('watch_items', []))}

DAILY MARKET SUMMARY:
{daily_view.get('summary', 'No summary available')}

Based on this comprehensive market intelligence and the predicted next 10-day price movements, provide an EXTENSIVE institutional-grade Bitcoin investment advisory that includes:

1. **Executive Summary & Market Overview** (200+ words)
2. **Investment Recommendation** (specific position sizes, entry/exit points, timeframes)
3. **Risk Assessment & Management** (detailed risk analysis, hedging strategies)
4. **Price Targets & Scenarios** (incorporating the 10-day predictions, multiple scenarios)
5. **Trading Strategy & Execution** (entry strategies, portfolio allocation, timing)
6. **Market Outlook & Catalysts** (short/medium/long-term outlook)
7. **Technical Analysis Integration** (support/resistance, momentum indicators)  
8. **Fundamental Analysis** (adoption trends, regulatory landscape, institutional flows)
9. **Risk-Reward Analysis** (expected returns, maximum drawdown, Sharpe ratios)
10. **Alternative Scenarios** (black swan events, regulatory changes)
11. **Portfolio Integration** (correlation with other assets, diversification)
12. **Actionable Investment Thesis** (clear rationale, conviction level)

Make your analysis comprehensive, data-driven, and suitable for institutional investors managing significant Bitcoin allocations. Use the next 10-day price predictions to inform your near-term tactical recommendations while maintaining focus on long-term strategic positioning."""

        return prompt
    
    def generate_investment_advisory(self, daily_data, max_retries: int = 3) -> Optional[str]:
        """Generate comprehensive investment advisory with robust timeout handling"""
        
        # Analyze 60-day price movements for better advisory (hidden from training prompts)
        next_60_day_prices = daily_data.get('next_60_day_prices', [])
        next_10_day_prices = daily_data.get('next_10_day_prices', [])
        
        if next_60_day_prices and next_10_day_prices:
            price_analysis = self.analyze_price_movements_for_better_advisory(next_10_day_prices, next_60_day_prices)
        else:
            price_analysis = {}
        
        # Create a simplified prompt for better API stability
        date_str = daily_data['date'].strftime('%Y-%m-%d') if hasattr(daily_data['date'], 'strftime') else str(daily_data['date'])
        
        for attempt in range(max_retries):
            try:
                # Use progressively simpler prompts if API keeps timing out
                if attempt == 0:
                    # Full prompt on first attempt
                    prompt = self.create_investment_advisory_prompt(daily_data)
                    max_tokens = 2000
                    timeout = 120
                elif attempt == 1:
                    # Shorter prompt on second attempt
                    prompt = self.create_simplified_advisory_prompt(daily_data)
                    max_tokens = 1000
                    timeout = 80
                else:
                    # Minimal prompt on final attempt
                    prompt = self.create_minimal_advisory_prompt(daily_data)
                    max_tokens = 600
                    timeout = 60
                
                print(f"🔄 Attempt {attempt + 1}: {len(prompt)} char prompt, {max_tokens} max tokens, {timeout}s timeout")
                
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.7,
                    max_tokens=max_tokens,
                    timeout=timeout,
                )
                
                advisory = response.choices[0].message.content.strip()
                print(f"✅ Success on attempt {attempt + 1}: Generated {len(advisory)} characters")
                return advisory
                
            except Exception as e:
                error_msg = str(e)[:100]
                if "timeout" in error_msg.lower():
                    print(f"⏰ Timeout on attempt {attempt + 1}: {error_msg}")
                else:
                    print(f"❌ Error on attempt {attempt + 1}: {error_msg}")
                
                if attempt < max_retries - 1:
                    wait_time = 5 + (attempt * 2)  # Shorter waits: 5s, 7s, 9s
                    print(f"   Waiting {wait_time}s before retry with simpler approach...")
                    time.sleep(wait_time)
                else:
                    print(f"❌ All attempts failed for {date_str}")
                    return None
        
        return None
    
    def create_simplified_advisory_prompt(self, daily_data):
        """Create a simplified prompt for better API reliability"""
        date_str = daily_data['date'].strftime('%Y-%m-%d') if hasattr(daily_data['date'], 'strftime') else str(daily_data['date'])
        next_10_day_prices = daily_data['next_10_day_prices']
        price_change = ((next_10_day_prices[-1] / next_10_day_prices[0]) - 1) * 100
        
        return f"""Bitcoin Investment Advisory for {date_str}

Market Data:
• News items analyzed: {daily_data['total_news_items']}
• High impact news: {daily_data['high_impact_count']}
• Market sentiment: {daily_data['bullish_ratio']:.1%} bullish, {daily_data['bearish_ratio']:.1%} bearish
• Next 10-day predicted price change: {price_change:+.2f}%

Provide a comprehensive Bitcoin investment recommendation covering:
1. Investment action (buy/sell/hold) with specific rationale
2. Risk assessment and key factors to watch
3. Price targets and timeline expectations
4. Portfolio allocation suggestions

Keep response focused but comprehensive (800-1200 words)."""
    
    def create_minimal_advisory_prompt(self, daily_data):
        """Create a minimal prompt for maximum API reliability"""
        date_str = daily_data['date'].strftime('%Y-%m-%d') if hasattr(daily_data['date'], 'strftime') else str(daily_data['date'])
        next_10_day_prices = daily_data['next_10_day_prices']
        price_change = ((next_10_day_prices[-1] / next_10_day_prices[0]) - 1) * 100
        
        return f"""Bitcoin advisory for {date_str}:
Market: {daily_data['total_news_items']} news items, {daily_data['bullish_ratio']:.0%} bullish sentiment
Prediction: {price_change:+.1f}% change in 10 days

Provide investment recommendation: action, rationale, risks, targets (400-600 words)."""

# Initialize the enhanced investment advisor
print("� Initializing Enhanced Bitcoin Investment Advisor with 60-day price analysis capability...")
investment_advisor = BitcoinInvestmentAdvisor(
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_BASE_URL,
    model=DEFAULT_MODEL
)
print("✅ Enhanced Investment Advisor with hidden 60-day analysis initialized successfully!")
print("📊 Features:")
print("   • Training prompts include ONLY 10-day price predictions")
print("   • 60-day price analysis used for better advisory generation (hidden from training)")
print("   • Enhanced strategic positioning based on longer-term price movements")
print("   • Improved risk assessment using extended price volatility patterns")

� Initializing Enhanced Bitcoin Investment Advisor with 60-day price analysis capability...
✅ Enhanced Investment Advisor with hidden 60-day analysis initialized successfully!
📊 Features:
   • Training prompts include ONLY 10-day price predictions
   • 60-day price analysis used for better advisory generation (hidden from training)
   • Enhanced strategic positioning based on longer-term price movements
   • Improved risk assessment using extended price volatility patterns


In [5]:
# Initialize the investment advisor if client is configured
if OPENAI_API_KEY:
    investment_advisor = BitcoinInvestmentAdvisor(OPENAI_API_KEY, OPENAI_BASE_URL, DEFAULT_MODEL)
    print("✅ Bitcoin Investment Advisor initialized successfully!")
else:
    investment_advisor = None
    print("ℹ️ Investment advisor not initialized. Set AI_PLATFORM_API_KEY to enable API calls.")

✅ Bitcoin Investment Advisor initialized successfully!


# 4. Generate Investment Advisory Dataset

Process daily market data to generate comprehensive investment recommendations using Gemini Flash.

In [6]:
# ROBUST API PROCESSING WITH TIMEOUT RECOVERY
print(f"🚀 ROBUST Bitcoin Investment Advisory Generation")
print("="*60)
print("�️ Features: Progressive timeout handling, fallback prompts, immediate results")

# Reinitialize advisor with optimized settings
investment_advisor = BitcoinInvestmentAdvisor(
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_BASE_URL,
    model=DEFAULT_MODEL
)

# Use small test batch first to validate approach
TEST_BATCH_SIZE = 3  # Start with 3 samples to test robustness
df_to_process = df_daily_investment.head(TEST_BATCH_SIZE).copy()
print(f"🧪 Processing {len(df_to_process)} samples with robust timeout handling")

advisory_samples = []
processing_stats = {
    'success': 0,
    'timeout': 0,
    'error': 0,
    'total': 0
}

def robust_advisory_worker(row_data, sample_num, total_samples):
    """Robust worker that handles timeouts gracefully"""
    
    date_str = row_data['date'].strftime('%Y-%m-%d') if hasattr(row_data['date'], 'strftime') else str(row_data['date'])
    print(f"\n� [{sample_num}/{total_samples}] Processing {date_str}")
    
    processing_stats['total'] += 1
    
    try:
        # Generate advisory with progressive fallback
        advisory = investment_advisor.generate_investment_advisory(row_data)
        
        if advisory and len(advisory) > 100:
            # Create the full prompt for training data
            try:
                full_prompt = investment_advisor.create_investment_advisory_prompt(row_data)
            except:
                # Fallback to simplified prompt if full prompt fails
                full_prompt = investment_advisor.create_simplified_advisory_prompt(row_data)
            
            result = {
                'date': date_str,
                'prompt': full_prompt,
                'response': advisory,
                'news_summary': f"{row_data['total_news_items']} news items ({row_data['high_impact_count']} high impact)",
                'market_sentiment': f"Bull: {row_data['bullish_ratio']:.1%}, Bear: {row_data['bearish_ratio']:.1%}",
                'next_10_day_change': f"{((row_data['next_10_day_prices'][-1] / row_data['next_10_day_prices'][0]) - 1) * 100:.2f}%",
                'next_60_day_change': f"{((row_data['next_60_day_prices'][-1] / row_data['next_60_day_prices'][0]) - 1) * 100:.2f}%",
                'advisory_length': len(advisory),
                'enhanced_features': 'Progressive timeout handling with fallback prompts'
            }
            
            processing_stats['success'] += 1
            success_rate = processing_stats['success'] / processing_stats['total'] * 100
            print(f"✅ SUCCESS {date_str}: {len(advisory)} chars (Success rate: {success_rate:.1f}%)")
            return result
            
        else:
            processing_stats['error'] += 1
            print(f"⚠️ FAILED {date_str}: Advisory too short or empty")
            
    except Exception as e:
        error_msg = str(e)
        if "timeout" in error_msg.lower():
            processing_stats['timeout'] += 1
            print(f"⏰ TIMEOUT {date_str}: All retry attempts exhausted")
        else:
            processing_stats['error'] += 1
            print(f"❌ ERROR {date_str}: {error_msg[:100]}")
    
    return None

print("⏳ Processing samples with robust timeout handling...")
print("💡 Strategy: Progressive prompt simplification, shorter timeouts, immediate fallbacks")

results = []
for i, (_, row) in enumerate(df_to_process.iterrows(), 1):
    result = robust_advisory_worker(row.to_dict(), i, len(df_to_process))
    results.append(result)
    
    # Show running statistics
    success_rate = processing_stats['success'] / processing_stats['total'] * 100
    timeout_rate = processing_stats['timeout'] / processing_stats['total'] * 100
    print(f"📈 Progress: {i}/{len(df_to_process)} | Success: {success_rate:.1f}% | Timeouts: {timeout_rate:.1f}%")

# Collect successful results
for result in results:
    if result:
        advisory_samples.append(result)

print(f"\n" + "="*60)
print(f"🎯 ROBUST PROCESSING RESULTS")
print(f"="*60)
print(f"✅ Successful advisories: {processing_stats['success']}")
print(f"⏰ Timeout failures: {processing_stats['timeout']}")
print(f"❌ Other errors: {processing_stats['error']}")
print(f"📊 Success rate: {processing_stats['success'] / processing_stats['total'] * 100:.1f}%")

# Convert to DataFrame for analysis
df_advisory_training = pd.DataFrame(advisory_samples)

if not df_advisory_training.empty:
    print(f"\n📊 Generated Dataset Statistics:")
    print(f"   Total advisories: {len(df_advisory_training)}")
    print(f"   Average length: {df_advisory_training['advisory_length'].mean():.0f} characters")
    print(f"   Min length: {df_advisory_training['advisory_length'].min():.0f} characters")
    print(f"   Max length: {df_advisory_training['advisory_length'].max():.0f} characters")
    
    # Show sample
    print(f"\n📄 Sample Advisory (Robust Processing):")
    sample = df_advisory_training.iloc[0]
    print(f"Date: {sample['date']}")
    print(f"Market Summary: {sample['news_summary']}")
    print(f"Sentiment: {sample['market_sentiment']}")
    print(f"10-Day Prediction: {sample['next_10_day_change']}")
    print(f"Advisory Length: {sample['advisory_length']:,} characters")
    
    # Verify training data integrity
    sample_prompt = sample['prompt']
    has_60_day = "60" in sample_prompt and "day" in sample_prompt.lower()
    print(f"\n🔍 Training Data Verification:")
    print(f"Prompt contains '60-day' references: {'❌ YES (ERROR!)' if has_60_day else '✅ NO (CORRECT!)'}")
    print(f"Prompt length: {len(sample_prompt):,} characters")
    
    if processing_stats['success'] > 0:
        print(f"\n� SUCCESS! Robust processing approach is working!")
        print(f"💡 To process full dataset: Change TEST_BATCH_SIZE to len(df_daily_investment)")
        print(f"� Current dataset size: {len(df_daily_investment)} total samples available")
    else:
        print(f"\n⚠️ No successful advisories generated - API issues persist")
        
else:
    print("❌ No advisories generated - all attempts failed due to API timeouts")
    print("💡 Recommendations:")
    print("   1. Try again later when API load is lower")
    print("   2. Check API endpoint status")
    print("   3. Consider using different model or API key")

🚀 ROBUST Bitcoin Investment Advisory Generation
�️ Features: Progressive timeout handling, fallback prompts, immediate results
🧪 Processing 3 samples with robust timeout handling
⏳ Processing samples with robust timeout handling...
💡 Strategy: Progressive prompt simplification, shorter timeouts, immediate fallbacks

� [1/3] Processing 2018-01-01
🔄 Attempt 1: 13051 char prompt, 1500 max tokens, 79s timeout


KeyboardInterrupt: 

# 5. Create and Upload Investment Advisory Dataset

Convert the investment advisory data into a HuggingFace dataset for training comprehensive investment advisory models.

In [ ]:
# PARALLEL PROCESSING - CONFIGURABLE WORKERS, AS_COMPLETED COLLECTION
print(f"🚀 PARALLEL Bitcoin Investment Advisory Generation - ALL SAMPLES")
print("="*70)
print("⚡ Features: configurable workers, robust timeout handling, full dataset processing")
print(f"� Processing ALL {len(df_daily_investment)} samples with parallelism")

from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import time

# Configuration
MAX_WORKERS = int(os.environ.get("ADVISOR_MAX_WORKERS", "40"))  # default 16 threads; adjust to API limits
TASK_TIMEOUT_SEC = int(os.environ.get("ADVISOR_TASK_TIMEOUT", "120"))
PROGRESS_EVERY = int(os.environ.get("ADVISOR_PROGRESS_EVERY", "25"))

# Thread-safe containers and counters  
advisory_samples_parallel = []
advisory_lock_parallel = threading.Lock()
processing_stats_parallel = {
    'success': 0,
    'timeout': 0, 
    'error': 0,
    'total': 0,
    'lock': threading.Lock()
}


def robust_advisory_worker_parallel(data_sample):
    """
    Thread-safe worker function for parallel advisory generation
    """
    with processing_stats_parallel['lock']:
        processing_stats_parallel['total'] += 1
        current_total = processing_stats_parallel['total']
    
    try:
        # Convert Timestamp to string for date display
        date_val = data_sample.get('date')
        if hasattr(date_val, 'strftime'):
            date_str = date_val.strftime('%Y-%m-%d')
        else:
            date_str = str(date_val)[:10]
        
        # Generate advisory using robust method
        advisory = investment_advisor.generate_investment_advisory(data_sample)
        
        if advisory and len(advisory.strip()) > 50:
            # Thread-safe append to results
            with advisory_lock_parallel:
                advisory_samples_parallel.append({
                    'date': date_str,
                    'advisory': advisory,
                    'success': True
                })
            
            with processing_stats_parallel['lock']:
                processing_stats_parallel['success'] += 1
                current_success = processing_stats_parallel['success']
            
            print(f"✅ [{current_total:4d}/{len(df_daily_investment)}] SUCCESS: {date_str} - Advisory generated ({len(advisory):,} chars)")
            return {'status': 'success', 'advisory': advisory}
        else:
            with processing_stats_parallel['lock']:
                processing_stats_parallel['error'] += 1
                current_error = processing_stats_parallel['error']
            
            print(f"❌ [{current_total:4d}/{len(df_daily_investment)}] ERROR: {date_str} - Invalid advisory response")
            return {'status': 'error', 'message': 'Invalid advisory'}
    
    except Exception as e:
        error_msg = str(e)
        date_str = str(data_sample.get('date', 'unknown'))[:10] if 'date' in data_sample else 'unknown'
        
        if 'timeout' in error_msg.lower() or 'time' in error_msg.lower():
            with processing_stats_parallel['lock']:
                processing_stats_parallel['timeout'] += 1
            print(f"⏰ [{current_total:4d}/{len(df_daily_investment)}] TIMEOUT: {date_str} - {error_msg}")
        else:
            with processing_stats_parallel['lock']:
                processing_stats_parallel['error'] += 1
            print(f"❌ [{current_total:4d}/{len(df_daily_investment)}] ERROR: {date_str} - {error_msg}")
        
        return {'status': 'error', 'message': error_msg}

# Execute parallel processing
print(f"🎯 Starting parallel processing with {MAX_WORKERS} workers...")
print("-" * 70)

start_time = time.time()

futures = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    for _, row in df_daily_investment.iterrows():
        futures.append(executor.submit(robust_advisory_worker_parallel, row.to_dict()))

    completed = 0
    for future in as_completed(futures, timeout=None):
        try:
            _ = future.result(timeout=TASK_TIMEOUT_SEC)
        except Exception as e:
            print(f"⚠️ Future execution failed: {str(e)}")
        finally:
            completed += 1
            if completed % PROGRESS_EVERY == 0:
                elapsed = time.time() - start_time
                rate = completed / max(elapsed, 1e-6)
                eta_minutes = (len(futures) - completed) / rate / 60 if rate > 0 else float('inf')
                with processing_stats_parallel['lock']:
                    current_success = processing_stats_parallel['success']
                    current_timeout = processing_stats_parallel['timeout']
                    current_error = processing_stats_parallel['error']
                    success_rate = (current_success / completed * 100) if completed > 0 else 0
                    timeout_rate = (current_timeout / completed * 100) if completed > 0 else 0
                print(
                    f"📈 Progress: {completed:4d}/{len(futures)} ({completed/len(futures)*100:.1f}%) | "
                    f"Success: {success_rate:.1f}% | Timeout: {timeout_rate:.1f}% | "
                    f"Rate: {rate:.1f}/sec | ETA: {eta_minutes:.1f}min"
                )

# Final results summary
elapsed_total = time.time() - start_time
print("\n" + "="*70)
print("🏁 PARALLEL PROCESSING COMPLETE!")
print("="*70)

with processing_stats_parallel['lock']:
    final_success = processing_stats_parallel['success']
    final_timeout = processing_stats_parallel['timeout']
    final_error = processing_stats_parallel['error']
    final_total = processing_stats_parallel['total']

print(f"📊 Final Statistics:")
print(f"   Total Processed: {final_total:,}")
print(f"   ✅ Successful: {final_success:,} ({(final_success/final_total*100) if final_total else 0:.1f}%)")
print(f"   ⏰ Timeouts: {final_timeout:,} ({(final_timeout/final_total*100) if final_total else 0:.1f}%)")
print(f"   ❌ Errors: {final_error:,} ({(final_error/final_total*100) if final_total else 0:.1f}%)")
print(f"   � Advisory Samples Generated: {len(advisory_samples_parallel):,}")
print(f"   ⏱️  Total Time: {elapsed_total:.1f} seconds ({elapsed_total/60:.1f} minutes)")
print(f"   �️ Workers Used: {MAX_WORKERS}")

if len(advisory_samples_parallel) > 0:
    print(f"\n✨ SUCCESS: Generated {len(advisory_samples_parallel):,} investment advisories!")
    print(f"📈 Sample advisory preview: {advisory_samples_parallel[0]['advisory'][:200]}...")
else:
    print(f"\n⚠️  WARNING: No successful advisories generated. Check API connectivity.")

🚀 PARALLEL Bitcoin Investment Advisory Generation - ALL SAMPLES
⚡ Features: configurable workers, robust timeout handling, full dataset processing
� Processing ALL 2437 samples with parallelism
🎯 Starting parallel processing with 40 workers...
----------------------------------------------------------------------
🔄 Attempt 1: 13051 char prompt, 2000 max tokens, 120s timeout
🔄 Attempt 1: 12886 char prompt, 2000 max tokens, 120s timeout
🔄 Attempt 1: 14247 char prompt, 2000 max tokens, 120s timeout
🔄 Attempt 1: 13374 char prompt, 2000 max tokens, 120s timeout
🔄 Attempt 1: 14241 char prompt, 2000 max tokens, 120s timeout
🔄 Attempt 1: 12899 char prompt, 2000 max tokens, 120s timeout
🔄 Attempt 1: 13798 char prompt, 2000 max tokens, 120s timeout
🔄 Attempt 1: 13123 char prompt, 2000 max tokens, 120s timeout
🔄 Attempt 1: 13510 char prompt, 2000 max tokens, 120s timeout
🔄 Attempt 1: 14389 char prompt, 2000 max tokens, 120s timeout
🔄 Attempt 1: 14029 char prompt, 2000 max tokens, 120s timeout
🔄 A

In [ ]:
# MONITOR PARALLEL PROCESSING PROGRESS
print("📊 Current Processing Status:")
print("="*50)


# Check if parallel processing variables exist
if 'processing_stats_parallel' in locals():
    with processing_stats_parallel['lock']:
        current_success = processing_stats_parallel['success'] 
        current_timeout = processing_stats_parallel['timeout']
        current_error = processing_stats_parallel['error']
        current_total = processing_stats_parallel['total']
    
    if current_total > 0:
        success_rate = (current_success / current_total * 100)
        timeout_rate = (current_timeout / current_total * 100) 
        error_rate = (current_error / current_total * 100)
        
        print(f"📈 Processed: {current_total:,} / {len(df_daily_investment):,} samples")
        print(f"✅ Success: {current_success:,} ({success_rate:.1f}%)")
        print(f"⏰ Timeout: {current_timeout:,} ({timeout_rate:.1f}%)")
        print(f"❌ Error: {current_error:,} ({error_rate:.1f}%)")
        print(f"📋 Advisory Samples: {len(advisory_samples_parallel):,}")
        
        if current_total == len(df_daily_investment):
            print("\n🎉 PROCESSING COMPLETE!")
        else:
            progress_pct = (current_total / len(df_daily_investment)) * 100
            print(f"⚡ Progress: {progress_pct:.1f}% complete")
    else:
        print("⏳ Processing starting...")
else:
    print("⚠️  Parallel processing not started yet")

# Check advisory samples if available
if 'advisory_samples_parallel' in locals() and len(advisory_samples_parallel) > 0:
    print(f"\n📝 Latest Advisory Sample ({advisory_samples_parallel[-1]['date'][:10]}):")
    print("-" * 50)
    latest_advisory = advisory_samples_parallel[-1]['advisory']
    print(latest_advisory[:300] + "..." if len(latest_advisory) > 300 else latest_advisory)

📊 Current Processing Status:
📈 Processed: 2,437 / 2,437 samples
✅ Success: 0 (0.0%)
⏰ Timeout: 0 (0.0%)
❌ Error: 2,437 (100.0%)
📋 Advisory Samples: 0

🎉 PROCESSING COMPLETE!
